#**Práctica 7: Redes Neuronales Artificiales Convolucionales 1 (Evaluación)**

Curso: Inteligencia Artificial para Ingenieros

Prof. Carlos Toro N. (carlos.toro.ing@gmail.com)

2022

## **Instrucciones de la evaluación**
- Entregar el notebook completamente ejecutado. Este está estructurado para que sea fácil de seguir y ejecutarlo paso a paso.
- Resolver las preguntas planteadas en el notebook, en particular el ejercicio final planteado.
- Comentar siempre sus resultados.

NOTA: asegúrese de que el hardware que usa el entorno de ejecución es una GPU. Verificar en Entorno de ejecución -> Cambiar tipo de entorno de ejecución.

##**Introducción**

En esta guía implementaremos redes neuronales: completamente conectada y convolucional, para resolver el problema de clasificación de dígitos escritos a mano y de imagenes de prendas de vestir.

**Descripción del dataset**: Los datos que usaremos en esta parte del laboratorio coresponde al dataset MNIST de dígitos escritos a mano, este contiene 60000 imágenes de entrenamiento y 10000 imágenes de prueba, cada una de 28x28 pixeles en escala de grises. El dataset fue creado por el investigador Yann LeCun y se encuentra descrito en todos lados, en particular, en su referencia original:

[Referencia del MNIST dataset](http://yann.lecun.com/exdb/mnist/).

<center>
    <img width="80%" src="https://miro.medium.com/max/1400/1*Riqqoa7vKHXnFHvaGfDFjA.webp">
</center>

**PREVIO:** Importaciones necesarios para la práctica

In [ ]:
# Imports
import tensorflow as tf
from tensorflow import keras
from keras import layers, optimizers, datasets
from matplotlib import pyplot as plt
import numpy as np

##1. Implementación de un clasificador con un perceptrón multicapa o red completamente conectada.

In [ ]:
# el dataset se encuentra disponible en todos lados, en particular en los datasets de tensorflow
help(datasets.mnist.load_data)

In [ ]:
# cargamos el dataset:
(x_train_raw, y_train_raw), (x_test_raw,y_test_raw) = datasets.mnist.load_data()

In [ ]:
# dimensiones de los datos
print(x_train_raw.shape,y_train_raw.shape)
print(x_test_raw.shape,y_test_raw.shape)

In [ ]:
# cómo se encuentran codificadas las etiquetas?
print(y_train_raw[0:10])# ejemplo, primeras diez muestras

In [ ]:
#Convirtiendo las etiquetas a one-hot encodes para entrenar la red yaque la capa de salida será una softmax
num_classes = 10
y_train = keras.utils.to_categorical(y_train_raw,num_classes)
y_test = keras.utils.to_categorical(y_test_raw,num_classes)

print(y_train[0])#cómo aparece ahora la salida?

### Preprocesamiento y visualización

In [ ]:
# mostremos algunas imagenes del dataset
plt.figure(figsize=(8,5))

for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_train_raw[i],cmap='gray')
    plt.axis("off")
    plt.title("Etiqueta: {} ".format(y_train_raw[i]))
plt.show()

In [ ]:
# Convertimos las imagenes de 28x28 pixeles a vectores de  784 elementos para pasarselos a la red
x_train = x_train_raw.reshape(60000,784)
x_test  = x_test_raw.reshape(10000,784)

# Preprocesamiento: Escalamos los valores de los pixeles para que estén en el rango de 0 a 1
x_train = x_train/255
x_test  = x_test/255

### Creación de un modelo DNN, Entrenamiento y Evaluación

In [ ]:
# Definición del modelo
model_DNN = keras.Sequential([
                              layers.Dense(100,activation='relu',input_dim=784),
                              layers.Dropout(0.2),
                              layers.Dense(num_classes,activation='softmax')
                             ])
model_DNN.summary()

In [ ]:
# Configuramos el entrenamiento
model_DNN.compile(optimizer = 'Adam', loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

# Entrenamos el modelo y guardamos el historial del entrenamiento para graficarlo
history = model_DNN.fit(x_train,y_train,
                        batch_size=128,
                        epochs=30,
                        validation_split = 0.1)

In [ ]:
# Gráficas del proceso de entrenamiento
from matplotlib import pyplot as plt
# Gráfica accuracy
plt.subplot(2, 1, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Acc. training')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Gráfica de la función de pérdida
plt.subplot(2, 1, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# Resultados finales en training
print('Loss final en training: ',history.history['loss'][-1])
print('Accuracy final en training: ',history.history['accuracy'][-1])

**Preguntas:** (**1.5 puntos**)
1. ¿Qué se observa del proceso de entrenamiento?
2. ¿Se podría haber detenido el proceso de entrenamiento antes de las 30 épocas sin afectar el resultado?, qué estrategia usaría para hacer esto. Impleméntela y compare los resultados con la evaluación del modelo en el conjunto de prueba.
3. Simular el proceso anterior sin hacer el escalamiento en los datos, ¿se obtienen mejores o peores resultados? (ojo con las variables como se están sobre-escribiendo, ejecutar todas las lineas que sean necesarias, siempre comprobar algunos valores/resultados con las funciones que ya conoce de numpy)

**Evaluamos en conjunto de test**

In [ ]:
# Evaluamos en conjunto de prueba
score = model_DNN.evaluate(x_test,y_test, verbose=0)

print('Loss en dataset de prueba: ',score[0])
print('Accuracy en dataset de prueba: ',score[1])

Mostremos algunas muestras de imágenes con etiquetas predichas por el modelo propuesto.

In [ ]:
# La siguiente linea permite devolver la clase predicha por el modelo,
# genera valores enteros coherentes con la codificación original
predicted_classes = np.argmax(model_DNN.predict(x_test), axis=-1)

# Verifiquemos que elementos fueron correcta e incorrectamente clasificados
correct_indices = np.nonzero(predicted_classes == y_test_raw)[0]
incorrect_indices = np.nonzero(predicted_classes != y_test_raw)[0]

In [ ]:
plt.figure(figsize=(10,5))
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[correct].reshape(28,28), cmap='gray')
    plt.title("Predicho {},\nClase Real {}".format(predicted_classes[correct], y_test_raw[correct]),fontdict={'color' : 'blue'})
    plt.axis('off')
plt.tight_layout()

plt.figure(figsize=(10,5))
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[incorrect].reshape(28,28), cmap='gray')
    plt.title("Predicho {},\nClase Real {}".format(predicted_classes[incorrect], y_test_raw[incorrect]),fontdict={'color' : 'red'})
    plt.axis('off')
plt.tight_layout()

## 2. Creación de modelo ocupando CNN

In [ ]:
#Creamos modelo de red CNN apilada con método secuencial
model_CNN = keras.Sequential([
           layers.Reshape(input_shape=(28*28,), target_shape=(28, 28, 1)),# transformamos las imagenes para indicar que están en escala de grises, es decir, un solo canal
           # Primera capa CONV + Pooling
           layers.Conv2D(filters=16,kernel_size=5,strides=2,padding='same',activation='relu', input_shape=(28,28,1)),
           layers.MaxPool2D(2),
           # Segunda capa CONV + Pooling
           layers.Conv2D(filters=32,kernel_size=3, strides=1,padding='same',activation='relu'),
           layers.MaxPool2D(2),
           # Operación de flattening
           layers.Flatten(),
           # Dropout
           layers.Dropout(0.25),
           # Capa completamente conectada
           layers.Dense(units=128,activation='relu'),
           layers.Dense(units=10,activation='softmax')
           ])

In [ ]:
model_CNN.summary()

In [ ]:
#configuración del entrenamiento
model_CNN.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

#entrenamiento
history = model_CNN.fit(x=x_train,y=y_train,
                        epochs=30,
                        batch_size=128,
                        validation_split = 0.1)

In [ ]:
# Gráficas del proceso de entrenamiento
from matplotlib import pyplot as plt
# Gráfica accuracy
plt.subplot(2, 1, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Acc. training')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Gráfica de la función de pérdida
plt.subplot(2, 1, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# Resultados finales en training
print('Loss final en training: ',history.history['loss'][-1])
print('Accuracy final en training: ',history.history['accuracy'][-1])

In [ ]:
# Evaluamos en conjunto de prueba
score = model_CNN.evaluate(x_test,y_test, verbose=0)

print('Loss en dataset de prueba: ',score[0])
print('Accuracy en dataset de prueba: ',score[1])

**Preguntas:** (**1.5 puntos**)
1. ¿Qué indica el argumento `padding='same'`? ¿Qué otro valor podemos configurar?
2. ¿Qué puede decir acerca del número de parámetros que necesitamos optimizar en ambos modelos, el número de capas usadas y los resultados?
3. ¿Qué se observa del proceso de entrenamiento? Habrá existido sobreajuste?
4. ¿Se podría haber detenido el proceso de entrenamiento antes de las 30 épocas sin afectar el resultado?, implementar la misma estrategia que en el caso del primer modelo.
5. Para el modelo óptimo encontrado en 4, mostrar la matriz de confusión en el conjunto de datos de prueba.


## **Ejercicio final**: Clasificación de imágenes dataset Fashion MNIST (3 puntos)

Este dataset contiene 70000 imágenes en escala de grises en 10 categorías. Las imágenes muestran articulos de ropa individuales de baja resolución (28x28 píxeles), como se muestra aquí:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figura 2.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>


[Referencia del dataset](https://www.tensorflow.org/tutorials/keras/classification?hl=es-419)

In [ ]:
# Imports
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

Importemos el dataset y hagamos algunas exploraciones iniciales

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Las etiquetas del dataset están codificadas con enteros de 0 a 9, las clases correspondientes son:


<table>
  <tr>
    <th>Etiqueta</th>
    <th>Clase</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

In [ ]:
# Definamos las clases
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Algunos datos que podemos extraer:

In [ ]:
# número de imágenes en el set de entreamiento
train_images.shape

In [ ]:
# número de etiquetas del conjunto de entrenamiento, claramente coinciden con el número de muestras
len(train_labels)

In [ ]:
# valores de las etiquetas primeras 10 muestras
train_labels[0:10]

In [ ]:
# set de test
test_images.shape

In [ ]:
# Ejemplo de una imagen del dataset
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.title(class_names[train_labels[0]])
plt.show()

In [ ]:
#cuántas imágenes hay por cada categoría?
np.unique(test_labels, return_counts=True)

### **Tareas a realizar:**
* Para resolver el problema, implementar el modelo de red neuronal LeNet-5, su arquitectura está descrita en la siguiente imagen:


<center>
    <img width="60%" src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cc/Comparison_image_neural_networks.svg/800px-Comparison_image_neural_networks.svg.png">
</center>

[Referencia modelo LeNet-5](https://en.wikipedia.org/wiki/LeNet).

Nota: si no se entrega la medida de stride en alguna capa, asumirlo como 1.
* Se recomienda implementar el entrenamiento con detención temprana.
* Graficar la matriz de confusión en el conjunto de prueba.
* Graficar muestras de imágenes bien y mal clasificadas con las etiquetas predichas por el modelo.

TIP: cuando implementen el modelo, mostrarlo con el método `summary()` para verificar que las dimensiones de salida coinciden con las de la imagen.

In [ ]:
#Su código aquí